# 第 6 章 因果推断难题

<style>p{text-indent:2em;2}</style>

似乎最具新闻价值的科学研究可信度最低，相反的，题目越枯燥，结果似乎值得信赖。为何存在这种普遍认为的负相关？似乎没有任何原因会导致人们关注的研究会导致不可信的结果。难道是因为热门话题吸引了越来越多的研究人员，就像苍蝇被蜂蜜的气味所吸引？

实际上，产生这种负相关的原因，来自于同行评审时选择的两项评判标准：论文的`新闻价值`和`可信度`。无论是授权评审还是期刊评审，如果编辑和审稿人都关心这两者的话，那么遴选行为本身天然就会导致负相关现象的出现。事实上，科学同行评审根本无法避免产生这种负相关，该事实也可以帮助我们进一步理解多元回归的危险性。

这里设计一个简单的模拟来说明。假设一个拨款审查小组收到 $200$ 份研究提案。在这些提案中，可信度（严谨、学术、合理性）和新闻价值（社会价值、公众兴趣）之间没有任何相关性；而小组成员也均平等地给可信度和新闻价值打分。然后根据总分对提案进行排名，并选择前 $10%$ 给予资助。

图 6.1 展示了模拟提案的完整样本，其中蓝色部分为最终的遴选结果。我根据选定的提案绘制了一条简单的线性回归线。显然在这个示例中存在负相关，斜率为 $-0.77$ 。这说明高分的遴选标准会导致选取准则之间存在负相关。为什么呢？大家思考一下，如果跨过门槛的唯一方法是获得高分，那么理论上在一个项目上获得高分通常会比在两个项目上都获得高分更容易。体现在图形上，也就意味着，在获得资助的提案中，最具新闻价值的研究实际上在可信度上会低于平均值；而最值得信赖的研究则在新闻价值上低于平均水平。也就是说，选择方式本身决定了负相关的结果，如果将这种负相关解释为“新闻价值”和“可信度”之间的反向因果关系，那就完全错了。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215141113-9e52.webp)

> **图 6.1**：为什么最具新闻价值的研究可能最不值得信赖。 $200$ 项研究提案按可信度和新闻价值相结合进行排名，遴选其中前 $10%$ 进行资助。图中显示，虽然选择前`可信度`和`新闻价值`之间不存在相关性（所有点呈随机状），但在选择后，两者呈现强的负相关，其相关性是-0.77 。

这种普遍现象早已为人所知，有时被称为伯克森悖论。但称其为 **选择失真效应** 可能更容易记住。一旦理解了这种效应，你会发现在任何地方都能看到它。为什么地段好的餐厅大多餐食比较差？一家餐食不太好的餐厅能够生存的唯一方法，就是其是否位于一个好位置。同样，美食餐厅即使在恶劣的地方也能生存。选择失真效应摧毁了你的城市。

那这和多元回归有什么关系吗？不幸的是，一切都有关系。前一章展示了多元回归的一些惊人威力。它可以消除虚假的相关性并消除掩蔽效应。这可能会鼓励这样一种观点：在有疑问时，只需将所有内容添加到模型中即可，让回归预言机来将其整理出来。

不幸的是，回归也无法解决。回归确实是神谕，但却是残酷的神谕。它用谜语说话，喜欢因为提出了不好的问题而惩罚我们。选择失真效应可能发生在多元回归中，因为添加预测变量的行为会在模型中引发统计性的选择，并导致毫无帮助的 *名称冲突偏差* 。这可能误导我们认为新闻价值和可信度之间存在负相关，而实际上这只是对某些变量进行调节的结果。这既是一个非常令人困惑的事实，也是为负责任地进行回归而需要理解的重要事实。

当简单地向回归模型中添加变量而没有明确因果模型概念时，可能发生可怕的事情，这正是本章和下一章讨论的重点。本章将探讨三种不同的危害：**多重共线性**、**后处理偏差**和**冲突偏差**。最后，我们将所有示例结合在一个框架中，该框架可以告诉我们，哪些变量必须和不可以添加到模型中，以便得出有效的推论。但是该框架并不会自动为我们做最重要的一步：给出一个有效的模型。

## 6.1 多重共线性

通常有许多潜在的预测变量可以添加到回归模型中。例如，在灵长类动物奶质数据的案例中，有 $7$ 个变量可用于预测其中任何一列。那为什么不直接构建一个包含所有 $7$ 项的模型呢？有几个危害。

首先从你最不担心的问题开始：**多重共线性**。

多重共线性意味着两个或多个预测变量之间存在强关联。原始相关性并不重要。重要的是取决于模型中的其他变量的关联。多重共线性的结果是后验分布似乎表明没有任何一个变量与结果能够非常可靠地相关联，尽管所有变量实际上都与结果密切相关。

这种令人沮丧的现象源于多元回归的工作细节。事实上，多重共线性并没有错。该模型可以很好地用于预测。你只会在试图理解它时感到沮丧。一旦理解了多重共线性，你就会更好地理解一般的回归模型。

让我们从一个简单的模拟开始。然后再次转向灵长类动物的奶质数据，看看真实数据集中的多重共线性。

### 6.1.1  多重共线的“腿”

想象一下，尝试用一个人的腿长作为预测变量，来预测其身高。身高肯定与腿长呈正相关，至少我们的模拟数据会假设这样。但如果你将两条腿的腿长都放入模型中，就会发生一些令人烦恼的事情。

下面的代码将模拟 $100$ 个人的身高和腿长。对于每个人，模拟数据先从高斯分布中抽取身高；然后在 $0.4$ 到 $0.5$ 区间的均匀分布中，随机抽取一个腿长过得比例；最后，考虑到每条腿都带有一点测量或发育误差，因此分别为左右腿长度设置了高斯误差项，这在真实人群中是典型的。

$$
\begin{align*}
height &\sim \cal{N}(10,2) \\
leg\_prop &\sim U(0.4,0.5)\\
leg\_left & \sim height \times leg\_prop + \cal{N}(0,0.02)\\
leg\_right & \sim height \times leg\_prop + \cal{N}(0,0.02)\\
\end{align*}
$$

现在让我们分析这些数据，使用两个预测变量 `leg_left` 和 `leg_right` 预测身高。在估计后验之前，先分析下预期：根据模拟数据，平均而言，腿长应当是身高的 45% 。因此，我们应该预期，体现腿长与身高之间的关联的系数 $\beta$ ，最终应当约为平均身高 ( 例中为 $10$ ) 除以平均身高 ( $4.5$ ) ，也就是 $10/4.5 ≈2.2$。现在让我们看看实际会是怎样。此处我将使用模糊先验，以确保先验对即将发生的事情不负责。

$$
\begin{align*}
height &\sim \cal{N}(\mu,\sigma) \\
\mu &\sim a + `bL` * \text{leg\_left} + br *\text{ leg\_right}\\
a &\sim \cal{N}(10,100)\\
`bL`  & \sim  \cal{N}(2,10)\\
br  & \sim  \cal{N}(2,10)\\
\sigma &\sim \text{Exp}(1.0)
\end{align*}
$$

上述模型在推断完成后，后验均值和标准差看起来很疯狂。下图显示了后验均值和 $89%$ 的可信区间，它让我们一眼就能看出这里出了什么问题：几个参数的均值都大大偏离了我们的预期。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215214609-b989.webp)

你可以继续尝试模拟几次，省略 `set.seed` 行。如果两条腿的长度几乎相同，而且身高与腿长的关系如此密切，后验分布怎么会如此奇怪呢？难道是后验推断出了问题？

不，后验推断工作很正常，其给出的后验分布正是我们问题的正确答案，出错的是问题本身。回想一下，多元线性回归回答了这样一个问题：在已知所有预测变量的情况下，了解每个预测变量的价值贡献。这体现在当前示例中，问题就变成了：在已经知道两条腿的长度后，每条腿长度的价值贡献如何？ 该问题是奇怪的，导致答案也同样奇怪，但似乎又合乎逻辑。

因此，此处给出的后验分布是符合问题的答案，它考虑了参数的每一种可能组合，并以模型和数据为条件，为每种组合分配了相对合理性。

查看 `bl` 和 `br` 的联合后验分布会有助于我们理解问题。如图 6.2 左侧所示。 `bl` 和 `br` 的后验分布高度相关，它们的所有合理值都位于一条狭窄的脊线上。当 `bl` 较大时，则 `br` 必须较小。由于两个腿长变量包含几乎完全相同的信息，而你又坚持将两者都包含在模型中，那么实际上将有无数种 `bl` 和 `br` 的组合会产生相同的预测。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215204945-e170.webp)

> 图 6.2 左：每条腿与身高的关联的后分布，来自模型 m6.1。由于两个变量包含几乎相同的信息，后验是负相关值的狭窄脊。右图：两个参数总和的后验分布集中在任何一条腿与身高的正确关联上。

一种思考此现象的路线是你估计了一个如下模型：

$$
\begin{aligned}
y_{i} & \sim \operatorname{Normal}\left(\mu_{i}, \sigma\right) \\
\mu_{i} &=\alpha+\beta_{1} x_{i}+\beta_{2} x_{i}
\end{aligned}
$$

变量 $y$ 是结果，如示例中的身高，而 $x$ 是单个预测变量，如示例中的腿长。这里 $x$ 使用了两次，这是使用两个腿长导致问题的完美示例。从泥人的角度来看，$\mu_{i}$ 的真实模型是：

$$
\mu_{i}=\alpha+\left(\beta_{1}+\beta_{2}\right) x_{i}
$$

我所做的只是从每个项中取出因子 $x_i$。参数 $β_1$ 和 $β_2$ 不能分开，因为它们永远不会分别影响均值 $μ$ 。只有它们的总和 $β_1+β_2$ 会影响 $μ$ 。这意味着后验分布最终会汇总 $β_1$ 和 $β_2$ 的组合，使得其和接近 $x$ 与 $y$ 的实际关联。

本模拟示例中的后验分布正是这样做的：它对 `b​​l` 和 `br` 的和产生了很好的估计。下面代码计算了两者之和的后验分布，并绘制了后验：

结果的密度图显示在图 6.2 的右侧。后验均值在 2 的右邻域中，略高于 2，并且标准差比任一分量 `bl` 或 `br` 小得多。如果你只用一个腿长变量来拟合回归，你会得到大致相同的后验均值。

这里的基本教训是：当两个预测变量强相关（以模型中的其他变量为条件）时，将两者都包含在模型中可能会导致混淆。在这种情况下，后验分布并没有错。它只是告诉你，你提出的问题无法用这些数据来回答。对于模型来说，它无法回答你的问题其实是一件好事。如果你只是对预测感兴趣，你会发现这个腿部模型能够做出很好的预测，它只是没有表明哪条腿更重要而已。

这个腿的例子清晰而可爱，但也纯粹是统计数据。我们并没有在这里提出任何严肃的因果问题。接下来让我们尝试一个更有趣的例子。

### 6.1.2 多重共线性的“牛奶”

在腿长的示例中，很容易看出在模型中包含两条腿有点傻。但在真实数据集中，我们很难预料到预测变量之间会发生冲突。因此可能会错误地解读后验分布，认为这两个预测变量都不重要。

在本节中，我们将通过真实数据查看此问题的一个示例。让我们回到灵长类动物的奶质数据。

在该示例中，我们关注 `perc.fat`（脂肪百分比）和 `perc.lactose`（乳糖百分比）变量，并使用这些量来模拟总能量含量 `kcal.per.g`。上面的代码已经将这三个变量归一化为 `F`、`L`、`K`。你将使用这三个变量来探索多重共线性的一种自然情况。请注意，这些列中没有缺失数据 `NA`，因此无需提取全部样本数据。但你可以放心，`quap` 函数不同于 `lm` 函数，它永远不会悄悄地丢弃样本。

首先将 `kcal.per.g` 建模为 `perc.fat` 和 `perc.lactose` 的函数，但采用两个一元回归。回顾第 5 章中对这些先验的讨论。

`bF` 和 `bL` 的后验分布本质上是彼此的镜像。 `bF` 的后验均值与 `bL` 的后验均值几乎互为相反数。而且两者都是狭窄的后验分布，几乎完全位于零的某一侧。考虑到每个预测因子与结果均有强关联，我们可以得出结论，这两个变量都是跨物种牛奶总能量的可靠预测因子。脂肪越多，牛奶中的卡路里就越多；乳糖越多，牛奶中的卡路里越少。但当我们将两个预测变量放在同一个回归模型中时会发生什么呢？

现在 `bF` 和 `bL` 的后验均值都更接近于零了，并且这两个参数的标准差是前面两个独立模型标准差的两倍。可以看出，这种现象与腿长示例中相同。发生的情况是变量 `perc.fat` 和 `perc.lactose` 包含许多相同的信息，以至于它们几乎是彼此的替代品。因此，当将两者包含在同一个回归中时，后验分布最终会画出同样合理的 `bF` 和 `bL` 的直线型联合分布。在脂肪和乳糖的情况下，这两个变量基本上形成一个单一的变化轴，查看这一点的最简单方法是使用`变量对的散点图`。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215204843-2539.webp)

> 图 6.3 来自灵长类动物奶质数据的总能量、脂肪百分比和乳糖百分比变量的成对图。脂肪百分比和乳糖百分比彼此呈强烈负相关，提供的信息大致相同。

我在图 6.3 中显示了随机变量两两之间的散点图矩阵。该矩阵对角线上为变量名，对角线外的每个散点图中，纵轴变量是该行标记的变量，横轴变量是该列标记的变量。例如，图 6.3 中第一行的两个散点图是 `kcal.per.g`（垂直）对 `perc.fat`（水平），然后是 `kcal.per.g`（垂直）对 `perc.lactose`（水平）。

请注意，图中明确显示，脂肪百分比与总能量呈正相关，而乳糖百分比与总能量呈负相关。现在查看中间行中最右侧的散点图。该图是脂肪百分比（垂直）与乳糖百分比（水平）的散点图，这些点几乎完全沿着一条直线排列，表明这两个变量之间呈非常强的负相关，以至于它们几乎是多余的。两者都有助于预测 `kcal.per.g`，但同时了解两者似乎并没有带来足够的好处。


在科技文献中，你可能会遇到各种应对多重共线性的方法，但其中很少有人会采取因果关系的观点。有些领域实际上直接教导学生，在拟合模型之前先检查变量两两之间的相关性，以识别或放弃高度相关的预测因子。虽然这种做法比较常见，但其实是错误的。**预测变量之间存在相关性并不是问题，更重要的是条件关联（`Conditional Associations`）而不是相关（ `Correlation`）**。但即使意识到这一点，最终的正确做法也将取决于导致共线性的原因，仅凭数据中的关联不足以决定做什么。

牛奶示例中可能存在的一种原因是：在哺乳动物中存在一种母性动物必须遵守的乳液成分中心权衡法则。如果一个物种经常哺乳，那么奶往往是水汪汪的，奶的能量需求较低，其含糖量高（乳糖）。相反，如果一个物种很少哺乳，那么必须在短时间内完成哺乳，奶的能量需要更高，其脂肪含量就会非常高。这意味着一个下面的因果图：

 
中心权衡法则决定了奶浓度  `D`的重要性，而我们并没有观测到这个变量，所以在图中它被圈起来了。我们只测定了脂肪 `F` 和乳糖 `L` ，并用 `F` 和 `L` 的组合去决定卡路里 `K`；此时如果我们可以测量 `D` ，或者有一个模型能够根据物种的其他要素来预测 `D`，那将会得到一个比当前回归模型更好的结果。

多重共线性问题是`拟合问题家族`的一员，该家族有时被称为**不可识别性（ `Non-Identifiability` ）**。当参数不可识别时，意味着数据和模型的结构无法估计参数的值。有时该问题是由模型编码错误引起的，但许多重要的模型，即使在编码完全正确的情况下，也都会存在不可识别或弱可识别的参数。大家应该逐步习惯于以下情况：**正确的模型不会天然地带来更加容易的推断**。

通常，无法保证可用数据中包含了有关参数的充分信息，如果其中关于参数的信息不足，则贝叶斯推断将返回与先验非常相似的后验分布。因此，将后验与先验进行比较可能是一个好主意，这是一种查看模型从数据中提取了多少信息的好方法。当后验和先验较为相似时，并不意味着计算是错误的，而可能代表出现了一个还没有意识到的问题。

## 6.2 处理后偏差

>  `处理后偏差` 可以解释为`包含处理后的效应变量而带来的偏差`，指在模型的预测变量中引入了经过处理以后的某些效应变量，进而导致整个模型出现了偏差。常出现在对治疗手段、处理方案的效果进行评估的场景中，往往会由于错误地将处理后的效应变量当做预测变量引入模型，进而导致整个模型出现偏差。

“担心因为忽略了预测变量而导致错误推断” 是一种很正常的心态，这类错误通常被称为 `遗漏变量偏差（ Omitted Variable Bias）`，前一章的例子说明了这一点。相反，“担心因为包含了某些变量而引起错误推断” 的情况比较少，但这种 `包含变量偏差（Included Variable Bias）`  却又是真实存在的。小心翼翼的随机实验和完全不受控的观测研究一样颇具危害性，盲目地将变量投入 `因果沙拉` 中不是一个好主意。

`包含变量偏差` 有多种形式。第一个是 `处理后偏差（Post-Treatment Bias）`，它在所有类型研究中都是一种风险。 “处理后” 一词实际上来自对实验设计的思考。例如：假设你正在温室中种植一些植物，想知道在经过不同抗真菌土壤处理后，植物生长的差异，因为植物上的真菌往往会减缓其生长。植物最初播种并发芽，测量得到其初始高度；然后应用不同的土壤处理手段；经过一段时间后，测量植物的高度和真菌的表现。因此，这里有四个感兴趣的变量：`初始高度`、`最终高度`、`处理手段` 和 `真菌的表现`。其中，`最终高度` 是感兴趣的结果，但模型中应该包含哪些预测变量呢？如果你的目标是对疗效进行因果推断，则不应包含 `真菌的表现` 这个变量，因为它是处理后的一种效应。

让我们模拟一些数据，看看当包含 `处理后变量` 时到底出了什么问题



### 6.2.1 先验的诞生

在设计模型时，假装你没有上面的数据生成过程会有所帮助。在真正的研究中，你不会知道真正的数据生成过程。但是你会有很多科学信息来指导模型构建。因此，让我们花点时间认真对待这个模拟分析。

我们知道时间 t = 1 的植物应该比时间 t = 0 的植物高，无论它们是在什么尺度上测量的。因此，如果我们将参数放在时间 t = 0 时的高度比例尺度上，而不是数据的绝对尺度上，我们可以更轻松地设置先验。为了使这更简单，让我们现在只关注高度变量，忽略预测变量。我们可能有一个线性模型，比如



其中 h0,i 是植物 i 在 t =0 时刻的高度，h1,i 是它在 t =1 时刻的高度，p 是衡量 h0,i 与 h1,i 的比例的参数。更准确地说，p = h1,i/h0,i。如果 p =1，则从时间 t =0 到时间 t =1，植物根本没有变化。如果 p = 2，则它的高度增加了一倍。因此，如果我们将 p 的先验集中在 1 上，则意味着高度没有变化的期望。这比我们知道的要少。但是我们应该允许 p 小于 1，以防实验出现严重错误并且我们杀死所有植物。我们还必须确保 p > 0，因为它是一个比例。回到第 4 章（第 96 页），我们使用了对数正态分布，因为它总是正的。让我们再用一个。如果我们使用 p∼Log-Normal(0,0.25)，先验分布看起来像

$$
\begin{align*}
h_{1, i} & \sim \operatorname{Normal}\left(\mu_{i}, \sigma\right) \\
\mu_{i} &=h_{0, i} \times p 
\end{align*}
$$


所以这个先验预期从 40% 的收缩到 50% 的增长。让我们拟合这个模型，这样你就可以看到它是如何衡量实验中的平均增长的

平均增长约 40%。现在包括治疗和真菌变量。我们将包括它们两个，遵循我们想要衡量治疗和真菌本身的影响的想法。这些变量的参数也将在比例范围内。它们将发生比例增长的变化。所以我们现在要制作 p 的线性模型。

$$
\begin{align*}
h_{1, i} & \sim \operatorname{Normal}\left(\mu_{i}, \sigma\right) \\
\mu_{i} &=h_{0, i} \times p \\
p &=\alpha+\beta_{T} T_{i}+\beta_{F} F_{i} \\
\alpha & \sim \log -\operatorname{Normal}(0,0.25) \\
\beta_{T} & \sim \operatorname{Normal}(0,0.5) \\
\beta_{F} & \sim \operatorname{Normal}(0,0.5) \\
\sigma & \sim \operatorname{Exponential}(1)
\end{align*}
$$

增长比例 p 现在是预测变量的函数。它看起来像任何其他线性模型。斜坡上的先验几乎可以肯定太平坦了。他们将 95% 的先前质量放在 -1（减少 100%）和 +1（增加 100%）之间，将三分之二的先前质量放在 -0.5 和 +0.5 之间。在我们完成本节后，你可能想要循环返回并尝试从这些先验中进行模拟。这是近似后验的代码：

a 参数与之前的 p 相同。它具有几乎相同的后验。 bt 的边缘后验，治疗效果，完全为零，间隔很窄。治疗与生长无关。然而，这种真菌似乎损害了生长。鉴于我们知道治疗很重要，因为我们以这种方式构建了模拟，这里发生了什么？ 

### 6.2.2.结果被阻止

问题是真菌主要是治疗的结果。这就是说真菌是治疗后的变量。因此，当我们控制真菌时，该模型隐含地回答了这个问题：一旦我们已经知道植物是否长出了真菌，土壤处理是否重要？答案是否定的，因为土壤处理通过减少真菌对生长产生影响。但我们实际上想知道，基于实验的设计，是处理对生长的影响。为了正确测量这一点，我们应该省略处理后的可变真菌。在这种情况下，推断是这样的

现在治疗的影响显然是积极的，理应如此。控制治疗前的差异（如初始高度 h0）是有意义的，这可能会掩盖治疗的因果影响。但包括治疗后变量实际上可以掩盖治疗本身。这并不意味着你不想要同时包含处理和真菌的模型。包括真菌使处理系数归零这一事实表明，该处理完全符合预期的原因。它告诉我们机制。但是关于治疗的正确推断仍然取决于省略治疗后变量。 

### 6.2.3.真菌和 `D-分离`。

从 DAG 的角度来看这个问题是有帮助的。在这种情况下，我将向你展示如何使用 dagitty R 包绘制它，因为我们现在将使用该包进行一些图形分析。

因此，处理 T 会影响真菌 F 的存在，真菌 F 会影响时间 1 H1 的植物高度。时间 1 的株高也受时间 0 的株高 H0 的影响。这就是我们的 DAG。当我们在模型中包含 F（治疗后效应）时，我们最终会阻断从治疗到结果的路径。

这是 DAG 的说法，一旦我们知道真菌状态，学习治疗不会告诉我们任何结果。一种更加 DAG 的说法是，对 F 的条件化会导致 d 分离。 “d”代表有向性。D-separation 意味着有向图中的某些变量与其他变量无关。没有连接它们的路径。在这种情况下，H1 与 T 分离，但仅当我们以 F 为条件时。以 F 为条件有效地阻止了有向路径 T → F → H1，使 T 和 H1 独立（d 分离）。在上一章中，当我们讨论隐含条件独立性时，你已经看到了这种陈述的符号 H1 ⊥⊥T|F。为什么会发生这种情况？ T 中没有关于 H1 的信息也没有在 F 中。所以一旦我们知道 F，学习 T 就不会提供关于 H1 的额外信息。你可以查询此 DAG 的隐含条件独立性

有三种。第三个是我们讨论的重点。但另外两个含义提供了测试 DAG 的方法。 F ⊥⊥ H0 和 H0 ⊥⊥ T 说的是原始植物高度 H0 不应该与处理 T 或真菌 F 相关联，前提是我们不以任何为条件。

显然，处理后变量的问题适用于对观察研究和实验都很好。但是在实验中，可以更容易地分辨哪些变量是处理前的，如 h0，哪些是后处理的，如真菌。在观察性研究中，更难知道。但是实验中也有很多陷阱。91 例如，对治疗后变量进行调节不仅可以使你误以为治疗无效。它也可以欺骗你认为它确实有效。考虑下面的 DAG

在此图中，处理 T 影响真菌 F，但真菌不影响植物生长。也许植物物种不会受到这种特殊真菌的困扰。新变量 M 是水分。它影响 H1 和 F。M 被圈起来表示它未被观察到。 H1 和 F 的任何未观察到的常见原因都可以——当然不一定是水分。 H1 对 T 的回归将显示处理与植物生长之间没有关联。但是如果我们在模型中包含 F，突然之间就会有关联。让我们试试吧。我将修改植物生长模拟，以便真菌对生长没有影响，但水分 M 会影响 H1 和 F

重新运行之前的模型，模型 m6.7 和 m6.8，现在使用 d2 中的数据。你会看到，包括真菌再次混淆了关于治疗的推断，这一次让它看起来好像对植物有帮助，即使它没有效果。

这个结果相当神秘。为什么M会有这种效果？下一部分是关于这样的效果。

## 6.3 冲突偏差

在本章开头，科学研究表明 “可信度和新闻价值之间存在负相关关系” 的必要条件是 “选择过程同时关注两者”。现在我想解释这种选择现象在统计模型中发生的原因。通常这种情况发生时，会严重扭曲我们的推断，此现象被称为**冲突偏差（Collider Bias）**。

让我们为这个例子考虑一个 `DAG` 。该模型中的 `可信度 ( T )` 和 `新闻价值 (N)` 与提交给资助审查小组的提案数量无关。但它们都影响 `资助选择（S）`。见下图：

两个箭头均指向 `S`  意味着它是一个冲突。核心概念很容易理解：当你以冲突为条件时，它会在其原因之间创建统计上的关联（但不一定是因果关系）。在这种情况下，一旦你了解到某个提案已被选中 ( `S` )，那么了解其可信度  (`T`)  也会提供有关其新闻价值 ( `N` ) 的信息。为什么呢？

因为，如果一个选定的提案可信度低，那么它必须具有高新闻价值，否则它不会得到资助。反过来也是一样，如果提案的新闻价值较低，我们会推断它的可信度必须高于平均水平，否则它也不会被选中。

这是在选定提案的总体中产生 `T` 和 `N` 之间负相关的信息现象。这意味着我们必须注意选择观测样本的过程，并且选择过程可能会扭曲变量之间的关联。但是，当你将冲突包含在预测变量内时，同样的现象也会在统计模型中产生误导性的关联。如果不小心，可能会做出错误的因果推断。

让我们考虑一个扩展的例子。 

### 6.3.1 虚假悲伤的冲突

考虑一下衰老如何影响幸福的问题。如果我们进行了一项对人们幸福程度进行评分的大型调查，那么年龄是否与幸福有关？如果有关，这种关联是因果关系吗？在这里，我想向你展示，如何通过控制看似合理的幸福混淆变量，导致对年龄影响的错误推断。

假设个人的平均幸福是一种在出生时就确定、不会随着年龄而改变的特征。虽然幸福不会随年龄而改变，但幸福确实会影响一个人的生活，其中给一个重要事件就是婚姻。更幸福的人更有可能结婚。此外，另一个影响婚姻的变量是年龄：你活得越久，最终结婚的可能性就越大。如果将这三个变量放在一起，就形成了一个因果模型：



`幸福（H）` 和 `年龄（A）` 都导致 `婚姻（M）` 。因此，`婚姻` 是一个冲突。即使 `幸福` 和`年龄`之间没有因果关系，如果以`婚姻`为条件 —— 即将`婚姻`作为回归的预测变量 —— 那么它就会导致 `年龄` 和 `幸福` 之间存在统计关联。这会误导我们认为 `幸福` 会随着 `年龄` 而变化，但实际上它是恒定的。

为了让你相信这一点，让我们再做一次模拟。这次将进行更高级的模拟，使用基于代理的 `衰老` 和 `婚姻` 模型来生成用于回归的模拟数据集。模拟设计如下：

 -  (1) 每年有 20 人出生，其幸福值分布均匀。
 -  (2) 每一年的每个人都会变老一岁，但幸福值不会改变。
 -  (3) 年满 18 岁时，个人可以结婚；每年结婚的几率与个人幸福值成正比。
 -  (4) 结婚后，个人保持婚姻状态。
 -  (5) 65岁后，个人离开样本 。
 
 我已经把这个算法写进了 `rethinking` 包中。你可以运行并收集 $1000$ 年的结果数据。 这些数据包括从出生到 $65$ 岁各个年龄段的 $1300$ 人。变量对应于上述 `DAG` 中的变量，模拟本身服从 `DAG`。
 
 图 6.4 中绘制了这些模拟数据，每个人显示为一个点。其中，填充点为已婚人士；水平方向为年龄，垂直方向为幸福，则最幸福的人在顶部。在 $18$ 岁时，这些人可以结婚，然后每年都会有越来越多的人结婚。因此，随着年龄增长，更多的人结婚了。但在所有年龄段，最幸福的人都更有可能结婚。

 ![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215210729-f56a.webp)

 > 图 6.4 模拟数据，假设幸福是均匀分布的，永远不会改变。每个点都是一个人。已婚个体用填充的蓝点显示。在 $18$ 岁以后的每个年龄段，最幸福的人都更有可能结婚。在晚年，更多的人倾向于结婚。此示例中，`婚姻状况`是`年龄`和`幸福`的冲突：$A → M ← H$。如果在回归中以婚姻为条件，它会误导我们认为`幸福`会随着`年龄`的增长而下降。

 假设你碰到这些数据并想分析年龄是否与幸福有关，而且你并不知道真正的因果模型。你有理由认为婚姻状况可能会令人困惑。如果已婚人士平均更幸福，那么你需要以婚姻状况为条件，推断年龄与幸福之间的关系。
 
 因此，让我们考虑一个多元回归模型，该模型旨在推断年龄对幸福的影响，同时控制婚姻状况。这只是一个简单的多元回归，就像本章和上一章中的其他回归一样。线性模型是这样的：

 $$
 \mu_i = \alpha_{MID[i]} + \beta_A A[i]
 $$

 其中 $MID[i]$ 是个体 $i$ 的 婚姻状况指标，$1$ 表示单身，$2$ 表示已婚。当我们使用多个截距，每个类别一个截距时，会比使用指示变量更容易设置先验。

 现在应该尽我们的职责并考虑先决条件。首先考虑斜率 $ β_A$，因为如何缩放预测变量 $A$ 将决定截距的含义。我们将只关注 $18$ 岁或 $18$ 岁以上的成人样本。想象一下，年龄和幸福感之间有很强的关联，幸福感在 $18$ 岁时达到最大值，在 $65$ 岁时达到最小值。如果我们重新调整年龄，对 $18$ 到 $65$ 的范围做归一化，可能会更容易一些。
 

 现在这个新变量 $A$ 的范围是从 $0$ 到 $1$，其中 $0$ 是 $18$ 岁，$1$ 是 $65$ 岁。在这些数据中，幸福是一个从 $-2$ 到 $+2$ 的任意尺度。因此，我们想象中的最强关系，将幸福从最大到最小，具有 $(2 -(-2))/1 = 4$ 的上升斜率。请记住，正态分布 $95%$ 的质量包含在 $2$ 个标准差内。因此，如果我们设置 $4$ 之前一半的标准差，我们就是说我们预计 $95%$ 的合理斜率小于最大强度。这不是一个很强的先验，但同样，它至少有助于将推理限制在现实范围内。现在是拦截。每个 $α$ 是当 $A_i=0$ 时 $μ_i$ 的值。在这种情况下，这意味着 $18$ 岁。所以我们需要让 $α$ 覆盖所有幸福分数。 $Normal(0,1)$ 会将 $95%$ 的质量放在 $-2$ 到 $+2$ 区间内。
 
 最后，让我们近似后验。我们还需要构建婚姻状况指数变量。我会这样做，然后立即提供 `quap` 代码。




该模型非常确定年龄与幸福感呈负相关。我们想将此模型的推论与忽略婚姻状况的模型进行比较。下面是边缘后验分布的比较：


相比之下，这个模型发现年龄和幸福之间没有关联

当我们以碰撞器为条件时，上面的模式正是我们应该期待的。碰撞器是婚姻状态。这是年龄和幸福的共同结果。结果，当我们以它为条件时，我们会在这两个原因之间产生虚假的关联。因此，对于 m6.9 模型，年龄与幸福感呈负相关。但这只是统计关联，而不是因果关联。一旦我们知道某人是否已婚，那么他们的年龄确实会提供有关他们幸福程度的信息。

你可以在图 6.4 中看到这一点。只考虑蓝点，已婚人士。仅在蓝点中，老年人的平均幸福感较低。这是因为随着时间的推移，越来越多的人结婚，因此已婚者的平均幸福感接近于人口平均值为零。现在只考虑开放点，未婚人士。平均幸福感会随着年龄的增长而下降，这也是事实。这是因为随着时间的推移，更快乐的人会迁移到已婚人群中。因此，在已婚和未婚亚群中，年龄与幸福感之间存在负相关关系。但在这两个亚群中，这都不能准确反映因果关系。

这个例子很容易辩解。婚姻不也应该影响幸福吗？如果幸福会随着年龄而变化呢？但这没有抓住重点。如果你没有因果模型，则无法从多元回归中进行推断。而且回归本身并没有提供证明因果模型所需的证据。相反，你需要一些科学。

### 6.3.2 闹鬼的 DAG

碰撞器偏差源于对常见结果的调节，如前一个示例所示。如果我们可以对图表进行排序，我们就可以避免它。但要看到潜在的冲突并不总是那么容易，因为可能存在无法衡量的原因。未测量的原因仍然会导致冲突偏差。所以我很遗憾地说，我们还必须考虑我们的 DAG 可能会闹鬼的可能性。

例如，假设我们想推断父母 (P) 和祖父母 (G) 对孩子 (C) 教育成就的直接影响。93 由于祖父母也可能影响他们自己孩子的教育，所以箭头 G →P .到目前为止，这听起来很容易。它的结构类似于我们上一章的离婚率示例：


但是假设对父母和他们的孩子有无法衡量的共同影响，例如社区，祖父母（他们现在住在西班牙南部海岸）没有这些影响。然后我们的 DAG 就会被未观察到的 U 所困扰

现在 P 是 G 和 U 的共同结果，所以如果我们以 P 为条件，它会偏向关于 G → C 的推断，即使我们永远不会测量 U。我不希望这个事实立即显而易见。因此，让我们爬过一个定量的例子。

首先，让我们模拟 200 个祖父母、父母和孩子的黑社会。这个模拟会很简单。我们只是将 DAG 投影为一系列隐含的函数关系。上面的 DAG 暗示：

 (1) P 是 G 和 U 的某个函数 
 
 (2) C 是 G、P 和 U 的某个函数 
 
 (3) G 和 U 不是任何其他已知变量的函数 
 
 我们可以做出这些暗示变成一个简单的模拟，使用 rnorm 生成模拟观察。但要做到这一点，我们需要比“某些功能”更精确一些。所以我会发明一些联想的力量

 这些参数就像回归模型中的斜率。请注意，我假设祖父母 G 对他们的孙子 C 的影响为零。该示例不依赖于该影响完全为零，但它会使课程更清晰。现在我们使用这些斜率来绘制随机观察

 我已经制作了邻域效应，U，二进制。这将使示例更易于理解。但是这个例子并不依赖于这个假设。其他行只是嵌入在 rnorm 中的线性模型。
 
 现在当我们试图推断祖父母的影响时会发生什么？由于祖父母的一些总影响通过父母，我们意识到我们需要控制父母。这是 C 对 P 和 G 的简单回归。通常我会建议将变量标准化，因为它使建立合理的先验更容易。但我将保持模拟数据的原始比例，这样你就可以看到上面斜率的推断会发生什么。如果我们改变了比例，我们不应该期望恢复这些值。但是如果我们不理会规模，我们应该能够恢复接近这些值的东西。所以我很抱歉在这里使用了模糊的先验，只是为了推进这个例子

 父母的推断效果看起来太大了，几乎是应有的两倍。这并不奇怪。 P 和 C 之间的一些相关性是由于 U，而模型不知​​道 U。这是一个简单的混淆。更令人惊讶的是，该模型确信祖父母的直接影响是伤害了他们的孙子孙女。回归没有错。但对该关联的因果解释将是。

 ![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215211310-1999.webp)

 > 图 6.5。未观察到的混淆和碰撞偏差。在这个例子中，祖父母只是通过父母间接地影响孙子孙女。然而，未观察到的邻里效应对父母和他们的孩子造成了一种错觉，认为祖父母会损害他们孙辈的教育。父母教育是一个碰撞器：一旦我们以它为条件，祖父母教育就会与孙子女教育负相关

 在这种情况下，碰撞器偏差是如何产生的？考虑图 6.5。请注意，我确实对变量进行了标准化以制作此图。所以轴上的单位是标准偏差。横轴是祖父母教育。纵向是孙子教育。有两个点云。蓝云包括居住在良好社区（U = 1）的孩子。黑云包括生活在恶劣社区 (U = -1) 的儿童。没有注意这两个点云都显示 G 和 C 之间的正相关。受过高等教育的祖父母有更多受过教育的孙子孙女，但这种影响完全是通过父母产生的。为什么？因为我们认为是这样。 G在模拟中的直接影响为零

那么，当我们以父母为条件时，负面关联是如何产生的呢？以父母为条件就像在具有类似教育的父母亚群中寻找一样。所以让我们尝试一下。在图 6.5 中，我用填充点突出显示了教育程度介于 45 至 60 分位之间的父母。这个范围没有什么特别的。它只是让现象更容易看到。现在，如果我们仅通过这些点绘制一条回归线，将 C 回归到 G，则斜率为负。我们的多元回归发现存在负关联。但它为什么存在？

它存在是因为，一旦我们知道 P，学习 G 就会无形地告诉我们关于邻域 U 的信息，而 U 与结果 C 相关联。我知道这令人困惑。正如我一直在说的，如果你感到困惑，那只是因为你在关注。因此，请考虑具有相同教育水平的两个不同父母，例如在第 50 个百分位的中位数。其中一位父母有一个受过高等教育的祖父母。另一个有一个受过良好教育的祖父母。在这个例子中，这些父母接受相同教育的唯一可能方式是他们住在不同类型的社区。我们看不到这些邻里效应——我们还没有测量它们，回想一下——但邻里的影响仍然传递给孩子 C。所以对于我们神话般的两个受教育程度相同的父母来说，祖父母受过高等教育的父母结束了和一个受教育程度较低的孩子在一起。祖父母受教育程度较低的孩子最终会得到受教育程度较高的孩子。 G 预测较低的 C。

未测量的 U 使 P 成为碰撞器，而对 P 的调节会产生碰撞器偏差。那么我们能做些什么呢？你必须测量 U。这是同样以 U 为条件的回归：

## 6.4 直面混乱

在本章和前一章中，有几个例子说明了如何使用多元回归来处理混乱。但同时我们也看到了多元回归也可能导致混乱，一些控制错误的变量会破坏推断。希望我已经成功地让你远离了 “仅仅将所有因素添加到模型中，期望回归能解决所有问题” 的简单想法，并且已经使你相信 “如果能够保持足够的小心和充分的知识，有效推断是可能的”。

但是哪些原理能够解释 “为何有时省略变量有时添加变量会产生相同的现象” ？是否还有其他因果怪物潜伏在那里，困扰着我们？我们需要一些原则来将这些案例统一在一起。

让我们将 `混乱` 定义为 “结果变量 $Y$ 与预测变量 $X$ 之间的关联不符合实际情况的任何上下文，如果已经通过实验确定了 $X$ 的值”。例如，假设我们感兴趣`教育（E） ` 与 `工资 （W）`  之间的关联。问题在于，在一个典型人群中，有许多未观测到的变量 `U` 会影响 `E` 和 `W` 。例如一个人住在哪里? 他们的父母是谁?他们的朋友是谁? 等。



如果我们在 `E` 上回归 `W`，因果效应的估计会被 `U` 混淆。它是混淆的，因为有两条路径连接 `E` 和 `W`：（1）$E →W$ 和（2）$E ←U →W$ .此处的“路径”仅表示你可以遍历以从一个变量到另一个变量的任何一系列变量，而忽略箭头的方向。这两条路径都在 E 和 W 之间建立了统计关联。但只有第一条路径是因果关系。第二条路径是非因果的。为什么？因为如果只有第二条路径存在，并且我们改变了 E，它不会改变 W。E 对 W 的任何因果影响仅在第一条路径上起作用。

我们如何隔离因果路径？最著名的解决方案是进行实验。如果我们可以随机分配教育水平，它会改变图表

操纵消除了 U 对 E 的影响。当我们自己决定教育时，未观察到的变量不会影响教育。随着 U 的影响从 E 中移除，这将移除路径 E ← U → W。它阻塞了第二条路径。一旦路径被阻塞，信息在 E 和 W 之间只有一种方式，然后测量 E 和 W 之间的关联将产生因果影响的有用度量。操纵消除了混淆，因为它阻止了 E 和 W 之间的另一条路径。

幸运的是，有统计方法可以获得相同的结果，而无需实际操纵 E。如何？最明显的是将 U 添加到模型中，以 U 为条件。为什么这也消除了混淆？因为它也阻断了 E 和 W 之间通过 U 的信息流。它阻断了第二条路径。

要理解为什么对 U 的条件化会阻塞路径 E ←U →W，请将此路径孤立地视为一个完整模型。一旦你学习了 U，学习 E 也不会给你提供关于 W 的额外信息。例如，假设 U 是一个地区的平均财富。富裕的地区有更好的学校，导致更多的教育 E，以及更高薪的工作，导致更高的工资 W。如果你不知道一个人居住的地区，了解这个人的教育 E 将提供有关他们的信息工资 W，因为 E 和 W 跨地区相关。但是当你知道一个人居住在哪个区域之后，假设 E 和 W 之间没有其他路径，那么学习 E 就不会告诉你关于 W 的更多信息。独立，以 U 为条件。

### 6.4.1 关闭后门。

**阻止某些预测变量 X 和某些结果 Y 之间的混淆路径被称为关闭后门**。我们不希望任何虚假关联通过进入预测变量 X 后面的非因果路径潜入。在上面的例子中，路径 E ←U →W 是一条后门路径，因为它以箭头并将 E 连接到 W。这条路径是非因果的——干预 E 不会通过这条路径导致 W 发生变化——但它仍然会在 E 和 W 之间产生关联。

现在有一些好消息。给定一个因果 DAG，总是可以说必须控制哪些变量（如果有）才能关闭所有后门路径。也可以说明不得控制哪些变量，以避免产生新的混淆。而且——一些更好的消息——只有四种类型的变量关系可以组合形成所有可能的路径。所以你真的只需要了解四件事以及信息如何在每件事中流动。我将定义四种类型的关系。然后我们会做一些例子。

![](https://gitee.com/XiShanSnow/imagebed/raw/master/images/stats-20211215212647-f8cd.webp)

> 图 6.6。四大元素混乱。任何有向无环图都是根据这些基本关系构建的。从左到右：X ⊥⊥ Y|Z 在 Fork 和 Pipe 中，X ̸⊥⊥ Y|Z 在碰撞器中，对后代 D 的调节就像对其父 Z 的调节

图 6.6 显示了每个元素关系的 DAG。每个 DAG，无论多么庞大和复杂，都是由这四种关系构建而成的。让我们考虑每个，从左到右。

(1) 第一种关系是我们上面使用的关系，一个叉：X ←Z →Y。这是经典的混乱因素。在分叉中，某些变量 Z 是 X 和 Y 的共同原因，从而在它们之间产生相关性。如果我们以 Z 为条件，那么 learningX 不会告诉我们关于 Y 的任何信息。X 和 Y 是独立的，以 Z 为条件。

(2) 第二种关系是管道：X →Z →Y。我们在讨论植物生长示例和后处理偏差时看到了这一点：处理 X 影响真菌 Z，影响生长 Y。如果我们现在以 Z 为条件，我们也阻止了从 X 到 Y 的路径。所以在叉和一个管道，中间变量的条件阻塞了路径。

(3) 第三种关系是碰撞器：X → Z ← Y。你在本章前面见过碰撞器。与其他两种类型的关系不同，在碰撞器中，X 和 Y 之间没有关联，除非你以 Z 为条件。以 Z 为条件，碰撞器变量，打开了路径。一旦路径打开，信息在X和Y之间流动。但是X和Y都不会对对方产生任何因果影响。

(4)第四种关系是后代关系。后代是受另一个变量影响的变量。以后代为条件的部分条件取决于其父项。在图 6.6 中最右侧的 DAG 中，对 D 的调节也会在较小程度上对 Z 进行调节。原因是 D 有一些关于 Z 的信息。在这个例子中，这将部分打开从 X 到 Y 的路径，因为 Z是一个碰撞器。但一般来说，条件作用于后代的后果取决于其父母的性质。后代是常见的，因为通常我们不能直接测量一个变量，而只有一些代理。

无论因果 DAG 看起来多么复杂，它总是由这四种关系构建而成。由于你知道如何打开和关闭每个变量，你（或你的计算机）可以确定需要包含或不包含哪些变量。秘诀如下：

（1）列出所有连接 X（感兴趣的潜在原因）和 Y（结果）的路径。

（2）根据每条路径是开放的还是封闭的进行分类。除非路径包含碰撞器，否则路径是开放的。

(3) 根据是否为后门路径对每条路径进行分类。后门路径有一个箭头进入 X。 

(4) 如果有任何打开的后门路径，决定关闭它的条件（如果可能的话）。

让我们考虑一些例子。

对 C 或 A 进行调节就足够了。从效率的角度来看，对 C 进行条件化是更好的主意，因为它也有助于提高 X → Y 估计的精度。请注意，对 U 进行条件化也是有效的。但是因为我们告诉 dagitty U 是未观察到的（见上面的代码），它没有在调整集中暗示它。

现在考虑通过 B 的第二条路径。这条路径确实包含一个碰撞器，U → B ← C。因此它已经关闭。这就是为什么上面的adjustmentSets没有提到B。事实上，如果我们在B上做条件，它会打开路径，造成混淆。然后我们对 X →Y 的推断会发生变化，但如果没有 DAG，我们将不知道这种变化是在帮助我们还是误导我们。包含变量会改变 X → Y 系数的事实并不总是意味着系数现在更好。你可以只使用冲突。

### 6.4.3 后门的`华夫饼`

作为最后一个例子，让我们回到第 5 章介绍中的 Waffle House 和离婚率相关性。我们将创建一个 DAG，用它来找到最小的协变量集，并用它来推导出可测试的含义DAG。这很重要，因为有时你确实可以测试你的 DAG 是否与证据一致。仅凭数据永远无法告诉我们 DAG 何时正确。但是数据可以告诉我们 DAG 何时出错。

我们对每个州的华夫饼屋数量对离婚率的总因果影响感兴趣。据推测，这两个变量之间的朴素相关是虚假的。阻止从华夫饼屋到离婚的后门路径的最小调整集是多少？让我们做一个图表

在这张图中，S 是一个州是否在美国南部，A 是结婚年龄中位数，M 是结婚率，W 是华夫饼屋的数量，D 是离婚率。该图假设南部各州的结婚年龄较低 (S → A)、直接结婚率 (S → M) 和通过结婚年龄介导的结婚率 (S → A → M) 以及更多华夫饼 (S → W)。结婚年龄和结婚率都会影响离婚。

W 和 D 之间有 3 条开放的后门路径。只需向后追溯，从 W 开始，到 D 结束。但请注意，所有后门路径都首先通过 S。因此我们可以通过以 S 为条件将它们全部关闭。仅此而已到它。你的电脑可以确认这个答案

我们可以控制 A 和 M 或单独控制 S。

这个 DAG 显然不能令人满意——它假设没有未观察到的混淆，这对于此类数据来说是不太可能的。但是我们仍然可以通过分析它来学习一些东西。虽然数据不能告诉我们一个图表是否正确，但它有时可以暗示一个图表是如何错误的。早些时候，我们讨论了条件独立性，这是模型的一些可测试含义。条件独立性是不相关的成对变量，一旦我们以其他变量集为条件。通过检查这些隐含的条件独立性，我们至少可以测试图的一些特征。

既然你知道了基本混淆，你就可以自己推导出任何 `DAG` 的条件独立性了。你可以使用在查找和关闭后门时学到的相同路径逻辑来查找条件独立性。你只需要关注一对变量，找到连接它们的所有路径，并找出是否有任何一组变量可以关闭它们。在一个大图中，这很麻烦，因为有很多对变量和可能有很多路径。但是你的电脑很擅长做这样的事。在这种情况下，存在三个隐含的条件独立性

将第一个读作“平均结婚年龄应该独立于 (_||_) 华夫饼屋，以 (|) 位于南部的州为条件。”第二，当我们同时以结婚年龄中位数、结婚率和华夫饼屋为条件时，离婚和在南方应该是独立的。最后，结婚率和华夫饼屋应该是独立的，条件是在南方。

在本章末尾的练习题中，我将要求你评估这些含义，并尝试评估华夫饼屋对离婚的因果影响。

## 6.4 总结

多元回归不是预言机，而只是一个傀儡。多元回归所描述的关系是有条件的关联，而不是因果效应。因此，需要来自模型外部的额外信息来理解它。本章介绍了一些常见问题的介绍性示例：**多重共线性**、**处理后偏差**和**冲突偏差**。这些问题可以在一个相干框架下被组织起来，并通过分析变量之间的假设因果关系来应对混淆。但应当清楚，在所有情况下，因果模型都存在于统计模型之外，而且难以测试。不过幸运的是，在没有实验的情况下，仍然有可能得出有效的因果推断。这无法进行实验的场景是一个好消息。